<a href="https://colab.research.google.com/github/iamanujkumar/PyTorch/blob/main/Building_ANN_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import matplotlib.pyplot as plt
import torch.optim as optim

In [3]:
torch.manual_seed(42)

In [4]:
df = pd.read_csv('/content/fmnist_small.csv')
df.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,pixel11,pixel12,pixel13,pixel14,pixel15,pixel16,pixel17,pixel18,pixel19,pixel20,pixel21,pixel22,pixel23,pixel24,pixel25,pixel26,pixel27,pixel28,pixel29,pixel30,pixel31,pixel32,pixel33,pixel34,pixel35,pixel36,pixel37,pixel38,pixel39,...,pixel745,pixel746,pixel747,pixel748,pixel749,pixel750,pixel751,pixel752,pixel753,pixel754,pixel755,pixel756,pixel757,pixel758,pixel759,pixel760,pixel761,pixel762,pixel763,pixel764,pixel765,pixel766,pixel767,pixel768,pixel769,pixel770,pixel771,pixel772,pixel773,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,125,72,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,3,0,2,0,0,210,228,228,233,0,0,0,0,0,0,0,0,0,31,81,133,184,201,190,117,0,0,2,1,0,7,0,50,205,196,213,165,0,0
1,7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,1,0,0,0,0,43,117,34,15,24,33,117,80,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,51,144,152,...,157,158,161,148,159,58,0,6,0,0,0,0,0,0,0,0,0,4,0,60,143,143,148,146,152,152,148,148,147,145,142,142,142,21,0,3,0,0,0,0
3,8,0,0,0,0,0,0,0,0,0,0,0,0,2,0,33,114,37,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,...,11,17,13,13,11,10,7,5,5,5,7,0,0,0,1,0,0,41,69,88,86,94,106,114,118,47,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,8,0,0,0,0,0,0,0,0,0,0,2,0,58,145,114,10,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,...,165,153,155,134,143,172,215,62,0,0,0,0,0,0,0,0,10,190,178,194,209,211,209,205,211,215,213,217,225,228,213,203,174,151,188,10,0,0,0,0


In [5]:
X = df.iloc[:,1:].values
y = df.iloc[:,0].values

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [15]:
X_train = X_train/255.0
X_test = X_test/255.0

In [16]:
X_train

array([[0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       ...,
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       [0.       , 0.       , 0.       , ..., 0.0627451, 0.       ,
        0.       ]])

In [18]:
class CustomDataset(Dataset):
  def __init__(self,features,labels):
    self.features = torch.tensor(features, dtype = torch.float32)
    self.labels = torch.tensor(labels,dtype=torch.long)

  def __len__(self):
    return len(self.features)

  def __getitem__(self,index):
    return self.features[index], self.labels[index]


In [44]:
train_dataset = CustomDataset(X_train, y_train)

In [45]:
test_dataset = CustomDataset(X_test, y_test)

In [46]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [47]:
class MyNN(nn.Module):
    def __init__(self,num_features):
      super().__init__()
      self.model = nn.Sequential(
          nn.Linear(num_features,128),
          nn.ReLU(),
          nn.Linear(128,64),
          nn.ReLU(),
          nn.Linear(64,10)
      )

    def forward(self,x):
      return self.model(x)

In [48]:
epochs = 100
learning_rate = 0.1

In [49]:
model = MyNN(X_train.shape[1])

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

In [50]:
#trainin look
for epoch in range(epochs):
  total_epoch_loss = 0
  for batch_features, batch_labels in train_loader:
    #farward pass
    outputs = model(batch_features)

    #loss calculate
    loss = criterion(outputs,batch_labels)
    optimizer.zero_grad()

    #backword pass
    loss.backward()

    #update grads
    optimizer.step()
    total_epoch_loss += loss.item()

  print(f"Epoch {epoch+1}/{epochs}, Loss: {total_epoch_loss/len(train_loader)}")

Epoch 1/100, Loss: 1.3962463776270548
Epoch 2/100, Loss: 0.8012886969248454
Epoch 3/100, Loss: 0.6638926458358765
Epoch 4/100, Loss: 0.5946846290429433
Epoch 5/100, Loss: 0.5435100906093915
Epoch 6/100, Loss: 0.5113831049203873
Epoch 7/100, Loss: 0.47809945364793144
Epoch 8/100, Loss: 0.4516102280219396
Epoch 9/100, Loss: 0.43864522129297256
Epoch 10/100, Loss: 0.4017714583873749
Epoch 11/100, Loss: 0.3805715382099152
Epoch 12/100, Loss: 0.36261646896600724
Epoch 13/100, Loss: 0.35419073075056073
Epoch 14/100, Loss: 0.3382030001282692
Epoch 15/100, Loss: 0.32054662172993026
Epoch 16/100, Loss: 0.3078585834304492
Epoch 17/100, Loss: 0.31101004995405673
Epoch 18/100, Loss: 0.28362751930952074
Epoch 19/100, Loss: 0.27185187314947445
Epoch 20/100, Loss: 0.25963711430629094
Epoch 21/100, Loss: 0.2651790730158488
Epoch 22/100, Loss: 0.2324234760304292
Epoch 23/100, Loss: 0.23592192667225997
Epoch 24/100, Loss: 0.2225617447992166
Epoch 25/100, Loss: 0.26421808193127316
Epoch 26/100, Loss: 0.2

In [51]:
model.eval()

MyNN(
  (model): Sequential(
    (0): Linear(in_features=784, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=10, bias=True)
  )
)

In [54]:
total = 0
currect = 0
with torch.no_grad():
  for batch_features, batch_labels in test_loader:
    outputs = model(batch_features)
    _, preducted = torch.max(outputs,1)

    total = total+ batch_labels.shape[0]

    currect = currect + (preducted == batch_labels).sum().item()

print(f"Test Accuracy: {currect/total}")

Test Accuracy: 0.84
